# # 🎯 Testing Non‐Linear Products

This notebook tests:
 
1. **Cap/Floorlets** (IBOR & Overnight)  
2. **Cap/Floor Streams**  
3. **Cap/Floor Portfolios**  
4. **Swaptions** (IBOR & Overnight)  
5. **Visitor output** for each product


In [1]:
from fixedincomelib.product.non_linear_products import (   
    ProductIborCapFloorlet,
    ProductOvernightCapFloorlet,
    CapFloorStream,
    ProductIborCapFloor,
    ProductOvernightCapFloor,
    ProductIborSwaption,
    ProductOvernightSwaption
)
from fixedincomelib.product.product_display_visitor import (
    IborCapFloorletVisitor,
    OvernightCapFloorletVisitor,
    IborCapFloorVisitor,
    OvernightCapFloorVisitor,
    IborSwaptionVisitor,
    OvernightSwaptionVisitor
)

# Common parameters
tNotional = 1_000_000
LONG = "LONG"
SHORT = "SHORT"

print("Setup complete.")

Setup complete.


## 2. Cap/Floorlet: IBOR 
Create a 3M IBOR caplet on 1M USD LIBOR, strike 2%, notional 1 000 000, long position.



In [2]:
caplet_ibor = ProductIborCapFloorlet(
    startDate="2025-07-01",
    endDate="2025-10-01",
    index="USD-LIBOR-BBA-1M",
    optionType="CAP",
    strike=0.02,
    notional=tNotional,
    longOrShort=LONG,
)
print("ProductIborCapFloorlet →")
print(caplet_ibor.accept(IborCapFloorletVisitor()))

ProductIborCapFloorlet →
      Attribute             Value
0  AccrualStart        2025-07-01
1    AccrualEnd        2025-10-01
2         Index  USD-LIBOR-BBA-1M
3    OptionType               CAP
4        Strike              0.02
5      Notional           1000000
6      Currency               USD
7   LongOrShort              LONG


In [3]:
floorlet_ibor = ProductIborCapFloorlet(
    startDate="2025-07-01",
    endDate="2025-10-01",
    index="USD-LIBOR-BBA-1M",
    optionType="FLOOR",
    strike=0.01,
    notional=1_000_000,
    longOrShort="SHORT",
)
print("ProductIborFloorlet →")
print(floorlet_ibor.accept(IborCapFloorletVisitor()), "\n")

ProductIborFloorlet →
      Attribute             Value
0  AccrualStart        2025-07-01
1    AccrualEnd        2025-10-01
2         Index  USD-LIBOR-BBA-1M
3    OptionType             FLOOR
4        Strike              0.01
5      Notional           1000000
6      Currency               USD
7   LongOrShort             SHORT 



## 3. Cap/Floorlet: Overnight 
Create a 1-month Overnight caplet on FedFunds, compounded, strike 1%, notional 500 000, short position.

In [4]:
caplet_ois = ProductOvernightCapFloorlet(
    effectiveDate="2025-07-01",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="COMPOUND",
    optionType="CAP",
    strike=0.015,
    notional=500_000,
    longOrShort="LONG",
)
print("ProductOvernightCapFloorlet →")
print(caplet_ois.accept(OvernightCapFloorletVisitor()), "\n")

ProductOvernightCapFloorlet →
       Attribute       Value
0  EffectiveDate  2025-07-01
1   MaturityDate  2025-10-01
2          Index     SOFR-1B
3    Compounding    COMPOUND
4     OptionType         CAP
5         Strike       0.015
6       Notional      500000
7       Currency         USD
8    LongOrShort        LONG 



In [5]:
floorlet_ois = ProductOvernightCapFloorlet(
    effectiveDate="2025-07-01",
    termOrEnd="2025-10-01",
    index="SOFR-1B",
    compounding="SIMPLE",
    optionType="FLOOR",
    strike=0.012,
    notional=500_000,
    longOrShort="SHORT",
)
print("ProductOvernightFloorlet →")
print(floorlet_ois.accept(OvernightCapFloorletVisitor()), "\n")

ProductOvernightFloorlet →
       Attribute       Value
0  EffectiveDate  2025-07-01
1   MaturityDate  2025-10-01
2          Index     SOFR-1B
3    Compounding      SIMPLE
4     OptionType       FLOOR
5         Strike       0.012
6       Notional      500000
7       Currency         USD
8    LongOrShort       SHORT 



## 4. Cap/Floor Stream

Build a quarterly (3M) cap stream from July 1 2025 to July 1 2026, strike 2.5%, notional 1 000 000.

In [6]:
stream = CapFloorStream(
    startDate="2025-07-01",
    endDate="2026-07-01",
    frequency="3M",
    iborIndex="USD-LIBOR-BBA-3M",
    optionType="CAP",
    strike=0.025,
    notional=1000000,
    longOrShort="LONG",
)
print("Num caplets:", stream.numProducts)

# %%
# Show first two caplets
for i in range(2):
    print("---")
    print(stream.element(i).accept(IborCapFloorletVisitor()))

Num caplets: 4
---
      Attribute             Value
0  AccrualStart        2025-07-01
1    AccrualEnd        2025-10-01
2         Index  USD-LIBOR-BBA-3M
3    OptionType               CAP
4        Strike             0.025
5      Notional           1000000
6      Currency               USD
7   LongOrShort              LONG
---
      Attribute             Value
0  AccrualStart        2025-10-01
1    AccrualEnd        2026-01-02
2         Index  USD-LIBOR-BBA-3M
3    OptionType               CAP
4        Strike             0.025
5      Notional           1000000
6      Currency               USD
7   LongOrShort              LONG


## 5. Cap/Floor Portfolio Wrappers
 
### 5.1 IBOR Cap/Floor 

Wrap the above stream in a `ProductIborCapFloor`.
 

In [7]:
cap = ProductIborCapFloor(
    effectiveDate="2025-07-01",
    maturityDate ="2026-07-01",
    frequency    ="3M",
    index        ="USD-LIBOR-BBA-3M",
    optionType   ="CAP",        # ← CAP instead of FLOOR
    strike       =0.015,
    notional     =2_000_000,
    longOrShort  ="LONG",       # usually LONG for a cap
)
print("ProductIborCapFloor (Cap) →")
print(cap.accept(IborCapFloorVisitor()))

ProductIborCapFloor (Cap) →
       Attribute             Value
0  EffectiveDate        2025-07-01
1   MaturityDate        2026-07-01
2          Index  USD-LIBOR-BBA-3M
3     OptionType               CAP
4       Notional           2000000
5       Currency               USD
6    LongOrShort              LONG
7     NumCaplets                 4


In [8]:
cap = ProductIborCapFloor(
    effectiveDate="2025-07-01",
    maturityDate ="2026-07-01",
    frequency    ="3M",
    index        ="USD-LIBOR-BBA-3M",
    optionType   ="FLOOR",
    strike       =0.015,
    notional     =2_000_000,
    longOrShort  ="SHORT",
)
print("ProductIborCapFloor (Floor)→")
print(cap.accept(IborCapFloorVisitor()))

ProductIborCapFloor (Floor)→
       Attribute             Value
0  EffectiveDate        2025-07-01
1   MaturityDate        2026-07-01
2          Index  USD-LIBOR-BBA-3M
3     OptionType             FLOOR
4       Notional           2000000
5       Currency               USD
6    LongOrShort             SHORT
7     NumCaplets                 4


## 5.2 Overnight Cap/Floor
 
Wrap a “compound OIS” stream in `ProductOvernightCapFloor`.

In [9]:
ois_cap = ProductOvernightCapFloor(
    effectiveDate="2025-07-01",
    maturityDate ="2026-07-01",
    frequency    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.015,
    notional     =1_000_000,
    longOrShort  ="LONG",
)
print("ProductOvernightCapFloor (Cap) →")
print(ois_cap.accept(OvernightCapFloorVisitor()))

ProductOvernightCapFloor (Cap) →
       Attribute       Value
0  EffectiveDate  2025-07-01
1   MaturityDate  2026-07-01
2          Index     SOFR-1B
3     OptionType         CAP
4    Compounding    COMPOUND
5       Notional     1000000
6       Currency         USD
7    LongOrShort        LONG
8     NumCaplets           4


In [10]:
ois_floor = ProductOvernightCapFloor(
    effectiveDate="2025-07-01",
    maturityDate ="2026-07-01",
    frequency    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="FLOOR",
    strike       =0.012,
    notional     =1_000_000,
    longOrShort  ="SHORT",
)
print("ProductOvernightCapFloor (Floor) →")
print(ois_floor.accept(OvernightCapFloorVisitor()))

ProductOvernightCapFloor (Floor) →
       Attribute       Value
0  EffectiveDate  2025-07-01
1   MaturityDate  2026-07-01
2          Index     SOFR-1B
3     OptionType       FLOOR
4    Compounding    COMPOUND
5       Notional     1000000
6       Currency         USD
7    LongOrShort       SHORT
8     NumCaplets           4


## 6. Swaptions

### 6.1 IBOR Swaption

European swaption on a 5Y quarterly USD‐LIBOR swap, strike 1.75%, notional 1 000 000, long.


In [11]:
swaption_ibor = ProductIborSwaption(
    optionExpiry="2025-12-01",
    swapStart    ="2026-01-01",
    swapEnd      ="2031-01-01",
    frequency    ="3M",
    iborIndex    ="USD-LIBOR-BBA-3M",   # full registry key
    strikeRate   =0.0175,
    notional     =1_000_000,
    longOrShort  ="LONG",
    optionType   ="PAYER" 
)
print("ProductIborSwaption →")
print(swaption_ibor.accept(IborSwaptionVisitor()))

ProductIborSwaption →
     Attribute             Value
0   ExpiryDate        2025-12-01
1    SwapStart        2026-01-01
2      SwapEnd        2031-01-01
3        Index  USD-LIBOR-BBA-3M
4    FixedRate            0.0175
5     Notional           1000000
6     Currency               USD
7  LongOrShort              LONG


### 6.2 Overnight Swaption

European swaption on a 2Y monthly OIS, strike 1%, notional 500 000, short.

In [12]:
swaption_ois = ProductOvernightSwaption(
    optionExpiry  ="2025-10-01",
    swapStart     ="2025-11-01",
    swapEnd       ="2027-11-01",
    frequency     ="1M",
    overnightIndex="FF-1B",
    strikeRate    =0.01,
    notional      =500_000,
    longOrShort   ="SHORT",
    optionType    ="RECEIVER"
)
print("ProductOvernightSwaption →")
print(swaption_ois.accept(OvernightSwaptionVisitor()))

ProductOvernightSwaption →
     Attribute       Value
0   ExpiryDate  2025-10-01
1    SwapStart  2025-11-01
2      SwapEnd  2027-11-01
3        Index       FF-1B
4    FixedRate        0.01
5     Notional      500000
6     Currency         USD
7  LongOrShort       SHORT
